In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

/home/nahyun-hong/miniconda3/envs/myenv/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
messages = [
    SystemMessage(content="system"),
    HumanMessage(content="Human1"),
    AIMessage(content="AI1"),
    HumanMessage(content="Human2"),
    AIMessage(content="AI2"),
    HumanMessage(content="Human3"),
    AIMessage(content="AI3"),
]

In [4]:
# 대화 많이 남기기
trimer = trim_messages(
    max_tokens=1000,
    token_counter=llm
)

trimer.invoke(messages)

[SystemMessage(content='system', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [6]:
# 대화 적게 남기기
trimer = trim_messages(
    max_tokens=30,
    token_counter=llm
)

trimer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [8]:
# 과거 대화를 남기고 최근 대화를 지우기
trimer = trim_messages(
    max_tokens=30,
    token_counter=llm,
    strategy="first"
)

trimer.invoke(messages)

[SystemMessage(content='system', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={})]

In [9]:
# 최근 대화를 남기고 과거 대화를 지우기
trimer = trim_messages(
    max_tokens=30,
    token_counter=llm,
    strategy="last"
)

trimer.invoke(messages)

[HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [10]:
# 최근 대화를 남기고 과거 대화를 지우는데 시스템 메시지 남기기
trimer = trim_messages(
    max_tokens=30,
    token_counter=llm,
    strategy="last",
    include_system=True
)

trimer.invoke(messages)

[SystemMessage(content='system', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [11]:
# 최근 대화를 남기고 과거 대화를 지우는데 시스템 메시지 남기고 Human Message 부터 시작하기
trimer = trim_messages(
    max_tokens=30,
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)

trimer.invoke(messages)

[SystemMessage(content='system', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [18]:
messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
    HumanMessage(content="오늘은 피자를 먹어야지!"),
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할거야?"),
    HumanMessage(content="내일은 수영을 가야지!"),
    AIMessage(content="수영이라니 정말 종은 운동이야. 수영장은 어디로 다녀?"),
    HumanMessage(content="주말에는 영화를 보러갈거야!"),
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?")
]

In [10]:
trimmer = trim_messages(
    max_tokens=120,
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)

In [11]:
trimmer.invoke(messages)

[SystemMessage(content='사용자의 질문에 2문자 이내로 짧게 대답해', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내일은 수영을 가야지!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수영이라니 정말 종은 운동이야. 수영장은 어디로 다녀?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='주말에는 영화를 보러갈거야!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?', additional_kwargs={}, response_metadata={})]

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = trimmer | llm | parser

In [19]:
chain.invoke(messages + [
    HumanMessage(content="오늘 뭘 먹는다고 했지?")
    ])

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 내일은 수영을 가야지!\nAI: 수영이라니 정말 종은 운동이야. 수영장은 어디로 다녀?\nHuman: 주말에는 영화를 보러갈거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 오늘 뭘 먹는다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.51s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "아직 정하지 않았어요! 무엇인가 추천해줄까요?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
         

'아직 정하지 않았어요! 무엇인가 추천해줄까요?'

In [20]:
chain.invoke(messages + [
    HumanMessage(content="주말에 내가 뭐하러 간다고 했지?")
])

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 주말에는 영화를 보러갈거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 주말에 내가 뭐하러 간다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "주말에 영화를 보러 간다고 했어! 어떤 영화를 볼 예정이야?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
       

'주말에 영화를 보러 간다고 했어! 어떤 영화를 볼 예정이야?'

In [16]:
import langchain
langchain.debug = True